# Webcam zeichnen
In diesem Notebook verwenden wir einen Handdetektor von MediaPipe sowie eine Webcam, um ein Bild zu zeichnen. MediaPipe bietet eine Reihe von Anwendungen im Bereich des maschinellen Lernens. Für weitere Informationen zu MediaPipe google das Projekt oder besuchen die MediaPipe Github Seite.

**Schritt 1:** Einbinden von Bibliotheken, um bestimmte Aufgaben ausführen zu können:

In [ ]:
import cv2
import mediapipe
import numpy as np

**Schritt 2:** Erstellen eines hand_detector-Objekts sowie eines webcam-Objekts:

In [ ]:
# Initialisierung des Handdetektors aus Mediapipe
hand_detector = mediapipe.solutions.hands.Hands()

# Einen image_counter einrichten, um die Anzahl der angezeigten Bilder zu zählen
image_counter = 0

# Eine Bildvariable für die Leinwand erstellen
canvas_image = None

# Ein Webcam-Objekt aus der Webcam mit der ID „0“ erstellen
camera_id = 0
webcam = cv2.VideoCapture(camera_id)

**Schritt 3:** Verwenden des Handdetektors, um ein Bild mit Ihrer Hand zu zeichnen:
* Drücke **q**, um die Bildklassifizierung zu beenden und die Kamera zu deaktivieren.
* Drücke **s**, um das Bild der Leinwand zu speichern.
* Drücke **x**, um das Bild auf der Leinwand zurückzusetzen.

In [ ]:
# Starte eine while-Schleife, die so lange läuft, wie die Bedingung „True“ (wahr) ist. 
# In diesem Fall läuft die Schleife unendlich lange, wenn sie nicht mit „break“ gestoppt wird.
while True:
    
    # Versuche ein Bild von der Webcam zu bekommen.
    # Bei Erfolg wird dies in der Variable „success“ und das Bild in der variable „image“ gespeichert.
    success, image = webcam.read()
    
    # Spiegeln des Bildes, sodass es wie in einem Spiegel richtig angezeigt wird
    image = cv2.flip(image, 1)
    
    if canvas_image is None:
        canvas_image = np.zeros_like(image)
    
    # Ausgeben einer Erfolgsmeldung wenn das Bild erfolgreich erhalten wurde
    print("Wurde das Bild erfolgreich erhalten? ->", success)
    
    # Anhalten der Schleife („break“), wenn das Bild nicht erfolgreich empfangen wurde.
    # Dies ist der Fall, wenn die Variable success „False“ (falsch) ist
    if success == False:
        break
    
    # Ausgeben des aktuellen Zählerstands
    print("Bildnummer anzeigen:", image_counter)
    
    # Handerkennung mit KI (mit Hilfe der Mediapipe-Bibliothek)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hand_detector.process(image_rgb)
    print(results.multi_hand_landmarks)
    
    if results.multi_hand_landmarks is not None:
        for hand_landmarks in results.multi_hand_landmarks:
            mediapipe.solutions.drawing_utils.draw_landmarks(image_rgb, hand_landmarks)
            
            finger_zero_x = int(hand_landmarks.landmark[8].x * canvas_image.shape[1])
            finger_zero_y = int(hand_landmarks.landmark[8].y * canvas_image.shape[0])
            
            image_rgb = cv2.drawMarker(image_rgb, (finger_zero_x, finger_zero_y), (0,255,0), cv2.MARKER_STAR, 15, 2)
            canvas_image = cv2.drawMarker(canvas_image, (finger_zero_x, finger_zero_y), (0,255,0), cv2.MARKER_DIAMOND, 3, 2)
    
    # Anzeigen des empfangenen Bildes in einem Fenster
    cv2.imshow("Zeichne etwas!", canvas_image)
    
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
    cv2.imshow("Handdetektor", image_bgr)
    
    # Auf unbestimmte Zeit auf einen Tastendruck auf der Tastatur warten.
    # Wenn die Taste gedrückt wird, die gedrückte Taste in der Variablen „pressed_key“ speichern und fortgefahren
    pressed_key = cv2.waitKey(1)
    
    # Wenn die gedrückte Taste die Taste q war, wird die Schleife angehalten.
    if pressed_key == ord("q"):
        break

    # Wenn die gedrückte Taste die Taste s war, wird ein Bild der Leinwand gespeichert.
    if pressed_key == ord("s"):
        cv2.imwrite("zeichnung.png", canvas_image)
    
    # Wenn die gedrückte Taste die X-Taste war, wird das Leinwandbild zurückgesetzt.
    if pressed_key == ord("x"):
        canvas_image = None
    
    # 1 zum Bildzähler addieren, um das Bild zu zählen
    image_counter = image_counter + 1


# Kamera deaktivieren und Fenster schließen
webcam.release()
cv2.destroyAllWindows()